### statically-typed language : java,C, C++
 컴파일시에 에러가 검출된다. 타입구분이 명확한것들을 정적타입 랭귀지 라함.
 
### dynamic-typed language : python, javascript, Ruby... 
 인터프리터 언어라고도한다. 타입에 딱히 신경쓰지않아도 개발이가능하며 개발속도 기본적으로 빠르다.
 
IDE에서 statical language 는 코드 suggestion이 잘 적용되지만, python같은 dynamic language 는 지원이 상대적으로 부족하다.

In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import attention
from collections import deque
import pickle
import pyreader
import numpy as np

In [2]:
data= 'data_samples/'
data_path='data_samples/mapping.map'
pattern = 'preprocess.part*'
hidden_size = 10
seq_length = 1
batch_size = 100
num_samples = 0
with open(data_path, "rb") as f:
        word_to_id = pickle.load(f)
vocab_size = len(word_to_id)
num_samples = 10
attention_num=5
max_attention=3
lambda_type='state'
keep_prob=0.9,
num_layer=1
max_grad_norm=3
learning_rate = 0.01
files = [y for x in os.walk(data) for y in iglob(os.path.join(x[0], pattern))]

### 전처리를 다음과같이 한다.

#### 변수명:번호 인데 아래와 같이 나온다.

In [3]:
current_file = deque(files).popleft()
with open(current_file, 'rb') as f:
    current_data = pickle.load(f)

### 예를들어 파일한개를 돌려보면..

In [4]:
inv_map = {v: k for k, v in word_to_id.items()} # 아이디값:변수명 으로 reverse시킴.
print(word_to_id['class'],inv_map[56])
for i in range(len(current_data)):
    inputs_raw = [inv_map[k] for k in current_data[i].inputs[0]]
#     inputs_raw = [inv_map[k] for k in current_data[i].identifier_usage[0]]
    inputs_raw= ' '.join(inputs_raw)
    targets_raw = [inv_map[k] for k in current_data[i].targets[0]]
    targets_raws = ' '.join(targets_raw)
print(inputs_raw)
print('--------------------------------------------------------------------------')
print(targets_raws)

56 class
class §OOV§ ) bind_layers ( = 
 §<dedent>§ §OOV§ 
 
 return cmd_definitions ) §NUM§ , §OOV§ , §OOV§ ( = 
 §<dedent>§ datetime 
 
 §<indent>§ return yield ) §NUM§ ( = 
 §<dedent>§ §OOV§ 
 if ) §NUM§ . ERROR in dsz §OOV§ , §OOV§ [ ( = 
 §<dedent>§ §OOV§ : ) §OOV§ OpsField ) §NUM§ . ERROR , §NUM§ . §OOV§ , var419 ) §NUM§ . §OOV§ ( , §NUM§ . §OOV§ , §NUM§ . §OOV§ , §NUM§ . §OOV§ , §NUM§ . §OOV§ , §NUM§ . §OOV§ ( ( 
 §<indent>§ try ) §NUM§ .
--------------------------------------------------------------------------
§OOV§ ) bind_layers ( = 
 §<dedent>§ §OOV§ 
 
 return cmd_definitions ) §NUM§ , §OOV§ , §OOV§ ( = 
 §<dedent>§ datetime 
 
 §<indent>§ return yield ) §NUM§ ( = 
 §<dedent>§ §OOV§ 
 if ) §NUM§ . ERROR in dsz §OOV§ , §OOV§ [ ( = 
 §<dedent>§ §OOV§ : ) §OOV§ OpsField ) §NUM§ . ERROR , §NUM§ . §OOV§ , var419 ) §NUM§ . §OOV§ ( , §NUM§ . §OOV§ , §NUM§ . §OOV§ , §NUM§ . §OOV§ , §NUM§ . §OOV§ , §NUM§ . §OOV§ ( ( 
 §<indent>§ try ) §NUM§ . ERROR


### 위와같이 normalise + indent와 패딩, out of vocabulary의 값들이  나온다.

### 실제로 집어넣을 값들은 숫자로 feeding한다.

In [5]:
current_data[i].inputs[0]

array([ 56,   1,   3, 210,   4,   8,   2,   9,   1,   2,   2,  17,  65,
         3,  11,   6,   1,   6,   1,   4,   8,   2,   9,  82,   2,   2,
        10,  17, 245,   3,  11,   4,   8,   2,   9,   1,   2,  16,   3,
        11,   5,  69,  20,  13,   1,   6,   1,  14,   4,   8,   2,   9,
         1,   7,   3,   1,  23,   3,  11,   5,  69,   6,  11,   5,   1,
         6, 627,   3,  11,   5,   1,   4,   6,  11,   5,   1,   6,  11,
         5,   1,   6,  11,   5,   1,   6,  11,   5,   1,   6,  11,   5,
         1,   4,   4,   2,  10,  52,   3,  11,   5], dtype=int64)

### 위와같이 넣게되어있다.

In [6]:
with tf.Graph().as_default(), tf.Session() as session:
    masks_ = tf.placeholder(tf.bool, [seq_length, batch_size, attention_num], name="masks")
    input_data_ = tf.placeholder(tf.int32, [seq_length, batch_size], name="inputs")
    targets_ = tf.placeholder(tf.float32, [seq_length, batch_size], name="targets")
    
    a = attention.AttentionModel(input_data=input_data_,
                                 targets=targets_,
                                 masks=masks_,
                                 is_training=True,
                                 attention_num= attention_num,
                                 batch_size=batch_size,
                                 hidden_size=hidden_size,
                                 num_samples=num_samples,
                                 seq_length=seq_length,
                                 vocab_size=vocab_size,
                                 lambda_type=lambda_type,
                                 max_attention=max_attention)
    
    session.run(tf.global_variables_initializer())
    loss = a.loss
    minimize = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    
    for i, seq_batch in enumerate(current_data):
        actual_lengths = seq_batch.actual_lengths
        identifier_usage = seq_batch.identifier_usage
        inputs = seq_batch.inputs
        masks = tf.transpose(seq_batch.masks,[0,2,1]).eval()
        targets = seq_batch.targets
    
        feed_dict = {input_data_:inputs, targets_:targets, masks_:masks}
        loss_, _ = session.run([loss, minimize], feed_dict= feed_dict)
        print('loss', loss_[0])

Constructing Attention Cell
Constructing Basic Model
Constructing Attention Model
loss 21.2759
loss 19.4153
loss 9.18101
loss 6.10612
loss 8.77651
loss 3.57297
loss 8.16632
loss 6.31346
loss 3.21252
loss 8.21451
loss 3.89573
loss 8.10054
loss 6.6893
loss 3.42429
loss 8.50933
loss 7.54288
loss 0.143933
loss 8.86742
loss 6.83924
loss 3.05971
loss 8.35973
loss 9.44518
loss 6.5567
loss 5.74271
loss 0.566878
loss 0.188752
loss 0.539596
loss 8.34385
loss 7.62577
loss 7.40092
loss 7.2646
loss 7.97451
loss 7.63953
loss 1.64735
loss 7.34622
loss 5.97534
loss 6.41659
loss 5.27918
loss 7.57405
loss 3.50322
loss 0.438991
loss 7.3593
loss 7.1364
loss 7.35785
loss 4.72692
loss 1.11368
loss 4.83158
loss 7.27637
loss 7.13104
loss 7.09045
loss 6.61949
loss 7.21376
loss 6.64696
loss 5.17231
loss 6.84217
loss 6.28417
loss 5.99351
loss 6.43868
loss 5.84769
loss 5.82647
loss 7.13647
loss 4.9484
loss 5.43872
loss 3.15637
loss 2.18567
loss 2.83212
loss 2.70068
loss 2.52188
loss 5.91708
loss 6.11253
loss 2.28

ValueError: Cannot feed value of shape (2, 100) for Tensor 'inputs:0', which has shape '(1, 100)'